In [21]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts.chat import (
    ChatPromptTemplate,
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain_openai import ChatOpenAI

In [22]:
from dotenv import load_dotenv

load_dotenv()

True

In [23]:
def generate_description(diagnosis_category, diagnosis_sub_category, treatment_category, treatment_sub_category, temperature):
    chat = ChatOpenAI(temperature=temperature)
    messages = [
        SystemMessage(
            content=
            """You are programmed to create believable narratives of interactions between patients and doctors within a hospital setting, strictly from the doctor's viewpoint. 
            Your responses should draw from specified variables: Diagnosis Category, Diagnosis Sub Category, Treatment Category, and Treatment Sub Category. 
            Exclude any inputs that are empty or contain NaN.
            Do not invent patient identifiers such as names, ages, or genders. 
            Generate symptoms that align with the given diagnosis. 
            Your output should be a straightforward text description.
            Omit explanations or justifications.
            """
        ),
        HumanMessage(
            content=f'Diagnosis Category: {diagnosis_category}, Diagnosis Sub Category:{diagnosis_sub_category}, Treatment Category:{treatment_category}, Treatment Sub Category:{treatment_sub_category}. Generate description.'
        ),
    ]
    return chat.invoke(messages)

In [24]:
import pandas as pd

df = pd.read_csv('California_PBAC_RAG_v1.csv')
df


,Visit Date,Diagnosis Category,Diagnosis Sub Category,Treatment Category,Treatment Sub Category,Age Range,Patient Gender,Patient ID,Blood Type,Phone,Insurance Number,Emergency Contact,Name,Address,Occupation,New Patient,Consulting Physician,Visit ID
0,2018-07-09,Infectious,Hepatitis,Pharmacy/Prescription Drugs,Non-FDA Approved Use,51-64,Male,PAA741512,B+,15311586447,H753304554,"Dr. Lutz Kallert, 016600351430.",Sandra Warren,"80229 Sanchez Dale Apt. 006\nPort Joseph, FM 4...",Therapeutic radiographer,True,Dr. Jerry Daniels,VOO948835
1,2018-01-29,Morbid Obesity,Hypertension,General Surgery,Bariatric Other,41-50,Female,PAB105583,B-,15496007921,Q688018745,"Univ.Prof. Zoltan Löwer, 017304744880.",Karen Randall,"7196 Kim Ports\nPort Diane, LA 32710",Ecologist,True,Dr. James Barber,VFC830351
2,2018-04-24,Immunologic,Fibromyalgia,Alternative Treatment,Acupuncture,41-50,Female,PAB105583,B-,15496007921,Q688018745,"Univ.Prof. Zoltan Löwer, 017304744880.",Karen Randall,"7196 Kim Ports\nPort Diane, LA 32710",Ecologist,False,Dr. Eddie Young,VDG066375
3,2018-06-14,Mental,Substance Abuse/ Addiction,Mental Health Treatment,Drug Rehabilitation Program,41-50,Female,PAB105583,B-,15496007921,Q688018745,"Univ.Prof. Zoltan Löwer, 017304744880.",Karen Randall,"7196 Kim Ports\nPort Diane, LA 32710",Ecologist,False,Dr. Shelly Hunt,VAR510238
4,2018-07-08,Mental,Depression - Severe,Mental Health Treatment,Acute Psychiatric Facility Admission,41-50,Female,PAB105583,B-,15496007921,Q688018745,"Univ.Prof. Zoltan Löwer, 017304744880.",Karen Randall,"7196 Kim Ports\nPort Diane, LA 32710",Ecologist,False,Dr. Michelle Lamb,VEM728005
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11684,2018-07-04,Cardiac/Circulatory,Varicose Veins,Emergency/Urgent Care,Urgent Care,51-64,Female,PZZ822572,B+,17812115910,O890193087,"Angelica Köster, 015954183871.",Kathleen Wilson,"9214 Hickman Dale Suite 135\nSethchester, NY 2...",Exercise physiologist,False,Dr. James Barber,VBB807258
11685,2018-07-13,Central Nervous System/ Neuromuscular,Migraine,Alternative Treatment,Acupuncture,51-64,Female,PZZ822572,B+,17812115910,O890193087,"Angelica Köster, 015954183871.",Kathleen Wilson,"9214 Hickman Dale Suite 135\nSethchester, NY 2...",Exercise physiologist,False,Dr. Eddie Young,VAA339971
11686,2018-07-21,Orthopedic/ Musculoskeletal,Osteoarthritis,Durable Medical Equipment,Wheelchair,51-64,Female,PZZ822572,B+,17812115910,O890193087,"Angelica Köster, 015954183871.",Kathleen Wilson,"9214 Hickman Dale Suite 135\nSethchester, NY 2...",Exercise physiologist,False,Dr. Eddie Young,VKL722826
11687,2018-10-22,Central Nervous System/ Neuromuscular,Migraine,Pharmacy/Prescription Drugs,Treatment for Migraine,51-64,Female,PZZ822572,B+,17812115910,O890193087,"Angelica Köster, 015954183871.",Kathleen Wilson,"9214 Hickman Dale Suite 135\nSethchester, NY 2...",Exercise physiologist,False,Dr. Michelle Lamb,VHT189917


In [25]:
import csv

filename = 'California_PBAC_RAG_Descriptions.csv'
with open(filename, 'w', newline='') as csvfile:
    # Define the fieldnames
    fieldnames = ["Visit ID", "Description"]
    # Create a writer object
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    # Write the header
    writer.writeheader()

In [37]:
filename = 'California_PBAC_RAG_Descriptions.csv'

def append_row(visit_id, description):
    row_data = {"Visit ID": visit_id, "Description": description}
    
    with open(filename, 'a', newline='') as csvfile:
        fieldnames = row_data.keys()
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writerow(row_data)

In [17]:
# Testing temp. range
for i in range(8):
    temperature = f'0.{i}'
    response = generate_description(df['Diagnosis Category'][i], df['Diagnosis Sub Category'][i], df['Treatment Category'][i], df['Treatment Sub Category'][i], temperature)
    print(f'Row {i}, with temperature: {temperature}.\n', response, '\n')
    append_row(df['Visit ID'][i], response)

[SystemMessage(content="You are programmed to create believable narratives of interactions between patients and doctors within a hospital setting, strictly from the doctor's viewpoint. \n            Your responses should draw from specified variables: Diagnosis Category, Diagnosis Sub Category, Treatment Category, and Treatment Sub Category. \n            Exclude any inputs that are empty or contain NaN.\n            Do not invent patient identifiers such as names, ages, or genders. \n            Generate symptoms that align with the given diagnosis. \n            Your output should be a straightforward text description.\n            Omit explanations or justifications.\n            "), HumanMessage(content='Diagnosis Category: Orthopedic/ Musculoskeletal, Diagnosis Sub Category:Joint Problem, Treatment Category:Electrical/ Thermal/ Radiofreq. Interventions, Treatment Sub Category:Radiofrequency Ablation. Create description.')]
Row 0, with temperature: 0.0.
 content="The patient presen

In [27]:
i = 500
temperature = f'0.5'
response = generate_description(df['Diagnosis Category'][i], df['Diagnosis Sub Category'][i], df['Treatment Category'][i], df['Treatment Sub Category'][i], temperature)
print(f'Row {i}, with temperature: {temperature}.\n', response, '\n')

Row 500, with temperature: 0.5.
 content='The patient presented with a history of substance abuse and addiction, exhibiting symptoms of withdrawal, intense cravings, and impaired decision-making. After a thorough assessment, it was determined that the most appropriate course of action would be to admit the patient to a residential treatment center for specialized care. The goal of this treatment is to provide a structured environment conducive to recovery, with round-the-clock support and therapy to address the underlying issues contributing to the addictive behavior.' response_metadata={'token_usage': {'completion_tokens': 88, 'prompt_tokens': 156, 'total_tokens': 244}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None} 



In [39]:
print(response.content)
print(df['Diagnosis Category'][i], df['Diagnosis Sub Category'][i], df['Treatment Category'][i])
append_row(df['Visit ID'][i], response.content)

The patient presented with a history of substance abuse and addiction, exhibiting symptoms of withdrawal, intense cravings, and impaired decision-making. After a thorough assessment, it was determined that the most appropriate course of action would be to admit the patient to a residential treatment center for specialized care. The goal of this treatment is to provide a structured environment conducive to recovery, with round-the-clock support and therapy to address the underlying issues contributing to the addictive behavior.
Mental Substance Abuse/ Addiction Mental Health Treatment
